In [1]:
import re
import pandas as pd
from pprint import pprint
import numpy as np
import itertools

In [2]:
def Kumiawase(array) -> np.asarray:
    return np.array([*map(list, itertools.product(*array.T.tolist()))][::-1])

In [3]:
def Kounyusu(maisu, kuchisu, kingaku: int=200):
    A = maisu // kuchisu
    B = kuchisu * A
    return B, maisu - B, f"{maisu * kingaku:,}"

In [4]:
def pctRank(data, ranknum=3):
    Result = []
    columns = list(data.keys())
    sereas = pd.Series(data.values.reshape(-1))
    for idx in sereas.value_counts().index.values:
        df_counttop = data == idx
        df_counttop = np.sum(df_counttop)
        df_counttop = (df_counttop / df_counttop.sum()).values
        Result.append(np.concatenate([[idx], df_counttop]))
    Result = np.vstack(Result)
    Result = pd.DataFrame(Result, columns=["number"] + columns)
    Result = np.vstack([Result.sort_values(key)["number"].values[-ranknum:].astype(int) for key in columns]).T
    return Result

In [5]:
chikan = re.compile("[年月日]")
df = pd.read_csv("loto7.csv")
df["date"] = pd.to_datetime(df["date"].map(lambda x: str(x).replace("年", "-").replace("月", "-").replace("日", "-")))
df = df.drop("bornas1", axis=1)
df = df.drop("bornas2", axis=1)
df = df.drop("count", axis=1)
df = df.drop_duplicates("date")
df = df.set_index("date")
df = df.dropna()
df = df.astype(int)
resampleDate, resampleValue = [], []
for D, V in list(df.resample("Q")):
    if len(V) == 0:
        continue
    resampleDate.append(D)
    resampleValue.append(V)

In [6]:
Result = np.vstack(list(map(lambda x: pctRank(x, 6), resampleValue)))
Result = pctRank(pd.DataFrame(Result, columns=["val1", "val2", "val3", "val4", "val5", "val6", "val7"]), 3)
Result

array([[ 3,  7, 11, 15, 24, 29, 35],
       [ 2,  6, 14, 17, 26, 33, 36],
       [ 1,  5, 12, 18, 28, 30, 37]])

In [7]:
kumi = Kumiawase(Result)
Row, Col = Result.shape
print(Kounyusu(Row**Col, 25))
rescount = 0
for idx in range(len(kumi) // 1000):
    try:
        k = kumi[idx * 1000: (idx + 1) * 1000]
        query = " | ".join(list(map(lambda x: " & ".join([f"val{idx + 1}=={val}" for idx, val in enumerate(x)]), k)))
        result = df.query(query)
        rescount += len(result)
    except:
        break
rescount

(2175, 12, '437,400')


0

In [8]:
rescount

0